## 1 softmax回归的理论模型

### 1.1 模型定义
本章节中，如无特别说明，向量均指列向量。

softmax回归是logistic回归（适用于2类分类）扩展到多类分类的结果。设标签$c \in \{1, ..., C \}$，对于样本$(\vec{x}, y)$，
softmax回归预测样本标签为$c$的概率为
$$
p(y=c|\vec{x}) = softmax(\vec{w}_c^\top \vec{x}) = \frac{\exp (\vec{w}_c^\top \vec{x})}{\sum_{c'=1}^C \exp (\vec{w}_{c'}^\top \vec{x})},
$$
因此softmax回归的预测结果为
$$
\hat{y} = argmax_{c=1}^C p(y=c|\vec{x}) = argmax_{c=1}^C \vec{w}_c^\top \vec{x}.
$$
本质上，softmax回归是一个单层神经网络，输出层为softmax层。

### 1.2 单样本的矢量计算表达式
为了更方便地定义torch tensor，假设所有向量均为行向量。

设$d$为样本特征个数且$\vec{x} \in \mathbb{R}^{1 \times d}$，$W \in \mathbb{R}^{d \times C}$为待学习的权重，$\vec{b} \in \mathbb{R}^{1 \times C}$为偏置，则对于样本$(\vec{x}^{(i)}, y^{(i)})$，softmax回归的矢量计算表达式为
$$
\hat{\vec{y}}^{(i)} = softmax (\vec{x}^{(i)} W + \vec{b}),
$$
其中$\hat{\vec{y}}^{(i)}$的各个元素反应了softmax回归预测各标签的概率。

### 1.3 多样本的矢量计算表达式

为了更方便地定义torch tensor，假设所有向量均为行向量。

令$X \in \mathbb{R}^{n \times d}$是$n$个样本的特征矩阵，则
$$
\hat{Y} = softmax (XW+b).
$$
PyTorch会自动开启广播模式。

### 1.4 参数学习
采用交叉熵损失函数，只关心正确类别的预测概率：
$$
l (W, \vec{b}) = - \frac{1}{N} \sum_{n=1}^N \sum_{c=1}^C y_c^{(n)} \log \hat{y}_c^{(n)}.
$$
其梯度的求导过程并不难，虽然略复杂。最终参数的更新公式为
$$
W_{t+1} \leftarrow W_t + \alpha \bigg( \frac{1}{|\mathcal{B}|} \sum_{n \in \mathcal{B}} \vec{x}^{(n)} \Big(\vec{y}^{(n)} - \hat{\vec{y}}^{(n)}_{W_t}\Big)^\top \bigg)
$$
$$
b_{t+1} \leftarrow b_t + \alpha \bigg( \frac{1}{|\mathcal{B}|} \sum_{n \in \mathcal{B}} \Big(\vec{y}^{(n)} - \hat{\vec{y}}^{(n)}_{b_t}\Big) \bigg)
$$
其中$\vec{y}^{(n)}$是一个向量，仅有true label位置对应的元素为1。